In [ ]:
from statistics import correlation

import kagglehub

path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

In [ ]:
import os

In [ ]:
%pwd

In [ ]:
import pandas as pd

In [ ]:
books=pd.read_csv(f"{path}/books.csv")

In [ ]:
books

In [ ]:
books.isnull().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
ax=plt.axes()

sns.heatmap(books.isna().transpose() , cbar=False,ax=ax)

plt.xlabel("Columns")
plt.ylabel("Missing values")

plt.show()

In [ ]:
import numpy as np

In [ ]:
books['missing_descriptions']=np.where(books['description'].isna() ,1,0)
books['age_of_books']=2024- books['published_year']

In [ ]:
columns_of_interset=['num_pages',"age_of_books","missing_descriptions","average_rating"]

correlation_matrix=books[columns_of_interset].corr(method='spearman') #for non-continous

sns.set_theme(style="white")
plt.figure(figsize=(10,10))
heatmap=sns.heatmap(correlation_matrix,annot=True,fmt=".2f",cmap="coolwarm",
                    cbar_kws={"label":"Spearman correlation"})

In [ ]:
books_missing=books[~books['description'].isna() &
      ~books['num_pages'].isna() &
      ~books['average_rating'].isna()&
      ~books['published_year'].isna()
]

In [ ]:
books_missing["description"]

In [ ]:
books_missing['categories'].value_counts().reset_index().sort_values("count",ascending=False)

In [ ]:
books_missing['words_in_description']=books_missing["description"].str.split().str.len()

In [ ]:
books_missing['words_in_description']

In [ ]:
plt.hist(x=books_missing['words_in_description'],bins=50)

In [ ]:
books_missing.loc[books_missing['words_in_description'].between(1,4), "description"] 

In [ ]:
books_missing.loc[books_missing['words_in_description'].between(25,34), "description"] 

In [ ]:
books_missing25_words=books_missing[books_missing['words_in_description']>25]

In [ ]:
len(books_missing25_words)

In [ ]:
books_missing25_words['title_and_subtitle'] =(
    np.where(books_missing25_words["subtitle"].isna() , books_missing25_words['title'],
    books_missing25_words[['title','subtitle']].astype(str).agg(": ".join,axis=1))
)

In [ ]:
books_missing25_words['title_and_subtitle']

In [ ]:
books_missing25_words.sample(5)

In [ ]:
books_missing25_words['description']=books_missing25_words['description'].str.lower()

In [ ]:
print(books_missing25_words['description'])

In [ ]:
import re
def clean_descriptions(text):
    pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]'
    pattern_broad = r'[^\w\s]'
    cleaned_text_1 = re.sub(pattern, '', text)
    cleaned_text_2 = re.sub(pattern_broad, '', cleaned_text_1)

    return cleaned_text_2



In [ ]:
books_missing25_words['description']

In [ ]:
books_missing25_words['description']=books_missing25_words['description'].apply(clean_descriptions)

In [ ]:
books_missing25_words['tagged_description'] = books_missing25_words[["isbn13" ,"description"]].astype(str).agg(" ".join,axis=1)

In [ ]:
books_missing25_words['tagged_description']

In [ ]:
books_missing25_words

In [ ]:
books_missing25_words.drop(["subtitle","missing_descriptions","age_of_books","words_in_description"],axis=1,inplace=True)

In [ ]:
books_missing25_words.to_csv("clean_book.csv",index=False)